In [1]:
def cross(A, B):
    """A 와 B 에 포함된 원소들의 교차곱 (cross product) 을 반환한다."""
    return [a+b for a in A for b in B]

digits   = '123456789'
rows     = 'ABCDEFGHI'
cols     = digits
squares  = cross(rows, cols)
unitlist = ([cross(rows, c) for c in cols] + [cross(r, cols) for r in rows] +
            [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')])
units = dict((s, [u for u in unitlist if s in u]) for s in squares)
peers = dict((s, set(sum(units[s],[]))-set([s]))for s in squares)

In [2]:
def test():
    """A set of unit tests."""
    assert len(squares) == 81
    assert len(unitlist) == 27
    assert all(len(units[s]) == 3 for s in squares)
    assert all(len(peers[s]) == 20 for s in squares)
    assert units['C2'] == [['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2'],
                           ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9'],
                           ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']]
    assert peers['C2'] == set(['A2', 'B2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2',
                               'C1', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9',
                               'A1', 'A3', 'B1', 'B3'])
    print('All tests pass.')


In [3]:
test()

All tests pass.


In [4]:
def parse_grid(grid):
    """텍스트 형태로 구성된 grid 가 주어질 때 {칸 이름: 숫자 목록} 꼴의 사전 형태로 변환한다. 
    만약 모순이 있으면 False 를 반환한다."""
    # 처음에는 모든 칸이 어떤 숫자도 가질 수 있도록 하고, 숫자가 쓰인
    # 칸을 발견할 때마다 해당 칸에 배정한다.
    values = dict((s, digits) for s in squares)
    for s,d in grid_values(grid).items():
        if d in digits and not assign(values, s, d):
            return False ## 칸 s 에 d 를 배정할 수 없는 경우.
    return values
    
def grid_values(grid):
    """주어진 grid 를 {square: char} 형태의 사전으로 변환한다."""
    chars = [c for c in grid if c in digits or c in '0.']
    assert len(chars) == 81
    return dict(zip(squares, chars))


In [5]:
def assign(values, s, d):
    """values[s] 에서 d 를 제외한 모든 값을 지우고 제약 조건을 전파한 뒤,
    변경된 values 를 반환한다. 만약 모순이 있으면 False 를 반환한다."""
    other_values = values[s].replace(d, '')
    if all(eliminate(values, s, d2) for d2 in other_values):
        return values
    else:
        return False

def eliminate(values, s, d):
    """values[s] 에서 d 를 지운다. 만약 두 전략 중 하나에 해당하면
    적절히 제약 조건을 전파하고, 변경된 values 를 반환한다. 만약 모순을
    발견하면 False 를 반환한다."""
    if d not in values[s]:
        return values ## 이미 지워진 경우
    values[s] = values[s].replace(d,'')
    ## 1. 어떤 빈 칸에 들어갈 수 있는 숫자가 하나밖에 없다면, 해당 칸의 이웃들에는 그 숫자가 들어갈 수 없다.
    if len(values[s]) == 0:
        return False ## 모순: 이제 s 에는 어떤 숫자도 들어갈 수 없다
    elif len(values[s]) == 1:
        d2 = values[s]
        if not all(eliminate(values, s2, d2) for s2 in peers[s]):
            return False
    ## 2. 한 단위에 어떤 숫자가 들어갈 수 있는 칸이 하나밖에 없다면, 거기에 그 숫자를 쓴다.
    for u in units[s]:
        dplaces = [s for s in u if d in values[s]]
    if len(dplaces) == 0:
        return False ## 모순: d 는 이제 들어갈 자리가 없다
    elif len(dplaces) == 1:
            # d 는 이제 u 단위 중에 들어갈 수 있는 곳이 한 군데밖에 없다: 거기에 넣는다
        if not assign(values, dplaces[0], d):
            return False
    return values


In [6]:
def display(values):
    "values 가 주어질 때 2차원 격자 형태로 출력한다."
    width = 1+max(len(values[s]) for s in squares)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '') for c in cols))
        if r in 'CF': print(line)
    print


In [7]:
grid1 = '003020600900305001001806400008102900700000008006708200002609500800203009005010300'

In [8]:
display(parse_grid(grid1))

4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 


In [9]:
grid2 = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'

In [10]:
display(parse_grid(grid2))

   4      1679   12679  |  139     2369    269   |   8      1239     5    
 26789     3    1256789 | 14589   24569   245689 | 12679    1249   124679 
  2689   15689   125689 |   7     234569  245689 | 12369   12349   123469 
------------------------+------------------------+------------------------
  3789     2     15789  |  3459   34579    4579  | 13579     6     13789  
  3679   15679   15679  |  359      8     25679  |   4     12359   12379  
 36789     4     56789  |  359      1     25679  | 23579   23589   23789  
------------------------+------------------------+------------------------
  289      89     289   |   6      459      3    |  1259     7     12489  
   5      6789     3    |   2      479      1    |   69     489     4689  
   1      6789     4    |  589     579     5789  | 23569   23589   23689  


In [11]:
def solve(grid): return search(parse_grid(grid))

def search(values):
    "깊이 우선 탐색과 제약 조건 전파를 이용해 모든 값들을 하나하나 시도한다."
    if values is False:
        return False ## 호출 이전에 실패한 경우
    if all(len(values[s]) == 1 for s in squares):
        return values ## 해결 성공!
    ## 아직 답을 못 찾은 칸 중 가장 후보의 수가 적은 칸 s 를 찾는다
    n,s = min((len(values[s]), s) for s in squares if len(values[s]) > 1)
    return some(search(assign(values.copy(), s, d))
        for d in values[s])

def some(seq):
    "seq 의 원소 중 False 가 아닌 것을 하나 반환한다."
    for e in seq:
        if e: return e
    return False

In [12]:
hard1  = '.....6....59.....82....8....45........3........6..3.54...325..6..................'

In [13]:
from sudoku import *

In [14]:
solve_all([hard1]) 

. . . |. . 6 |. . . 
. 5 9 |. . . |. . 8 
2 . . |. . 8 |. . . 
------+------+------
. 4 5 |. . . |. . . 
. . 3 |. . . |. . . 
. . 6 |. . 3 |. 5 4 
------+------+------
. . . |3 2 5 |. . 6 
. . . |. . . |. . . 
. . . |. . . |. . . 
4 3 8 |7 9 6 |2 1 5 
6 5 9 |1 3 2 |4 7 8 
2 7 1 |4 5 8 |6 9 3 
------+------+------
8 4 5 |2 1 9 |3 6 7 
7 1 3 |5 6 4 |8 2 9 
9 2 6 |8 7 3 |1 5 4 
------+------+------
1 9 4 |3 2 5 |7 8 6 
3 6 2 |9 8 7 |5 4 1 
5 8 7 |6 4 1 |9 3 2 
(44.02 seconds)

